In [2]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib as plt
import os


<p>Image normalization: consistent resolution, normalizing pixel values <br>
data augmentation 

</p>

In [29]:
desired_height = 500
desired_width = 500


parent_folder_path = r"raw data"
output_parent_folder = r"processed data"

for subfolder_name in os.listdir(parent_folder_path):
    subfolder_path = os.path.join(parent_folder_path, subfolder_name)

    # Check if the item in the parent folder is a subfolder
    if os.path.isdir(subfolder_path):
        # Create a new directory
        new_name = "processed_" + subfolder_name
        output_folder_path = os.path.join(output_parent_folder, new_name)
        os.makedirs(output_folder_path, exist_ok=True)


        for filename in os.listdir(subfolder_path):

            if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_path = os.path.join(subfolder_path, filename)
                image = cv.imread(image_path)

                resized_image = cv.resize(image, (desired_width, desired_height))

                # Save the image
                output_path = os.path.join(output_folder_path, subfolder_name, filename)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                cv.imwrite(output_path, resized_image)
